In [1]:
import sys, subprocess

print("Python executable:", sys.executable)

# Show the packages installed in THIS exact environment
subprocess.run([sys.executable, "-m", "pip", "show", "plotly"], check=False)
subprocess.run([sys.executable, "-m", "pip", "show", "kaleido"], check=False)

Python executable: c:\Users\Vanja\anaconda3\python.exe


CompletedProcess(args=['c:\\Users\\Vanja\\anaconda3\\python.exe', '-m', 'pip', 'show', 'kaleido'], returncode=0)

In [2]:
import sys, subprocess

subprocess.run([sys.executable, "-m", "pip", "install", "-U", "plotly>=6.1.1"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-U", "kaleido>=1.2.0"], check=True)

print("Done. Now restart the kernel and run imports again.")

Done. Now restart the kernel and run imports again.


In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import kaleido

import plotly
from importlib.metadata import version, PackageNotFoundError

print("Plotly:", plotly.__version__)

try:
    print("Kaleido:", version("kaleido"))
except PackageNotFoundError:
    print("Kaleido: not installed in this environment")

c:\Users\Vanja\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Vanja\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\Vanja\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Plotly: 6.5.2
Kaleido: 1.2.0


In [4]:
import openpyxl
print(openpyxl.__version__)

3.1.5


# 1. Overview (total number of metrics - clusters)

For the first one (the pie chart as general overview), please include the two new "clusters". The data needed is in the blue table in the excel sheet and a pie chart next to it.

In [50]:
# ==========================
# 1. Load and prepare data
# ==========================
# Adjust the path 
file_path = "MetricLibrary3.xlsx"
df = pd.read_excel(file_path, sheet_name='M_C')

df = df.rename(columns={
    "Cluster": "cluster",
    
    "Number of metrics cluster": "n_metrics"
})

df = df[["cluster", "n_metrics"]]
df.head()

,cluster,n_metrics
0,Timeliness,4
1,Informativeness,4
2,Representativeness,8
3,Measurement Process,17
4,Consistency,2


In [51]:
# Compute total and percentages
total_metrics = df["n_metrics"].sum()
df["percentage"] = df["n_metrics"] / total_metrics * 100
df.head()

,cluster,n_metrics,percentage
0,Timeliness,4,6.666667
1,Informativeness,4,6.666667
2,Representativeness,8,13.333333
3,Measurement Process,17,28.333333
4,Consistency,2,3.333333


In [52]:
# Text label to show on the pie slices:
# ClusterName N (%)
df["label_text"] = df.apply(
    lambda r: f"{r['cluster']} {int(r['n_metrics'])} ({r['percentage']:.1f}%)",
    axis=1
)

In [53]:
# Text INSIDE slices → only "N ( % )"
df["slice_text"] = df.apply(
    lambda r: f"{int(r['n_metrics'])} ({r['percentage']:.1f}%)",
    axis=1
)

In [54]:
# ==========================
# 2. Define cluster colors
# ==========================
cluster_colors = {
    "Measurement Process":      "#9C9EFF",
    "Consistency":              "#C8BAF3",
    "Representativeness":       "#77EBA8",
    "Informativeness":          "#51AAFF",
    "Timeliness":               "#73D3FC",
    "Distribution Metrics":     "#DB9ECF",
    "Correlation Coefficients": "#F79B7C"
}

In [55]:
#data type colors
data_colors = {
    "Images": "#448888",
    "Time-series": "#629797",
    "Tabular": "#7da7a7",
    "Text": "#b3c6c6",
    "Multi-modal": "#ced6d6",
}

In [56]:
# Optional: seaborn style for the rest of your figures
sns.set_style("whitegrid")


In [57]:
import numpy as np
import plotly.express as px

# Define your custom cluster order
desired_order = [
    "Measurement Process",
    "Consistency",
    "Representativeness",
    "Informativeness",
    "Timeliness",
    "Distribution Metrics",
    "Correlation Coefficients"
]

df["cluster"] = pd.Categorical(df["cluster"], categories=desired_order, ordered=True)
df = df.sort_values("cluster")

# Create the 2-line text (keep <br> for inside; outside works too)
df["slice_text_multiline"] = df.apply(
    lambda r: f"{int(r['n_metrics'])}<br>({r['percentage']:.1f}%)",
    axis=1
)

# --- choose when to push labels outside ---
# Anything below this percent goes outside (tune 4–8 depending on crowding)
OUTSIDE_THRESHOLD_PCT = 6.0

is_small = df["percentage"] < OUTSIDE_THRESHOLD_PCT

# Pull out only the small slices (helps leader lines + avoids collisions)
pull = np.where(is_small, 0.08, 0.0)

fig = px.pie(
    df,
    names="cluster",
    values="n_metrics",
    color="cluster",
    color_discrete_map=cluster_colors,
    category_orders={"cluster": desired_order}
)

fig.update_traces(
    sort=False,
    text=df["slice_text_multiline"],
    textinfo="text",
    textfont=dict(size=26),  # ✅ force label font size

    # ✅ mixed strategy:
    textposition=np.where(is_small, "outside", "inside"),

    # inside label orientation
    insidetextorientation="horizontal",

    # help outside labels with leader lines
    pull=pull,

    hovertemplate=(
        "<b>%{label}</b><br>"
        "N = %{value}<br>"
        "% = %{percent:.1%}<extra></extra>"
    ),
)

fig.update_layout(
    title=None,
    showlegend=True,
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=10, b=10),
    font=dict(size=22),

    # ✅ IMPORTANT: do NOT allow Plotly to shrink text
    #uniformtext_minsize=24,
    #uniformtext_mode="show"
)

fig.show()


In [58]:
import numpy as np
import pandas as pd
import plotly.express as px

# Define your custom cluster order
desired_order = [
    "Measurement Process",
    "Consistency",
    "Representativeness",
    "Informativeness",
    "Timeliness",
    "Distribution Metrics",
    "Correlation Coefficients"
]

df["cluster"] = pd.Categorical(df["cluster"], categories=desired_order, ordered=True)
df = df.sort_values("cluster")

# ✅ text: numbers only (no %)
df["slice_text"] = df["n_metrics"].astype(int).astype(str)

fig = px.pie(
    df,
    names="cluster",
    values="n_metrics",
    color="cluster",
    color_discrete_map=cluster_colors,
    category_orders={"cluster": desired_order}
)

fig.update_traces(
    sort=False,
    text=df["slice_text"],
    textinfo="text",
    textposition="inside",                 # ✅ force inside
    insidetextorientation="horizontal",
    textfont=dict(size=26),

    hovertemplate=(
        "<b>%{label}</b><br>"
        "N = %{value}<extra></extra>"      # ✅ hover shows only N
    ),
)

# ==========================
# ✅ Move pie left (no overlap with legend)
# ==========================
# Pie position controls (0..1): smaller x1 moves pie left, leaving space for legend
PIE_X0, PIE_X1 = 0.00, 0.72   # <- adjust PIE_X1 (e.g. 0.65–0.80)
PIE_Y0, PIE_Y1 = 0.00, 1.00

fig.update_traces(domain=dict(x=[PIE_X0, PIE_X1], y=[PIE_Y0, PIE_Y1]))

fig.update_layout(
    title=None,
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=10, b=10),
    font=dict(size=26),

    showlegend=True,
    legend=dict(
        x=0.78, y=0.5,                      # legend position (to the right)
        xanchor="left",
        yanchor="middle",
        title=dict(
        text="Cluster",      # 👈 legend title text
        font=dict(size=26)   # optional
        )
)
)

fig.show()


In [59]:
fig.write_image("Fig1_Metrics_overview3.png", scale=3)
fig.write_image("Fig1_Metrics_overview3.svg", scale=3)
fig.write_image("Fig1_Metrics_overview3.pdf", scale=3)

In [31]:
import numpy as np
import plotly.express as px

# Define your custom cluster order
desired_order = [
    "Measurement Process",
    "Consistency",
    "Representativeness",
    "Informativeness",
    "Timeliness",
    "Distribution Metrics",
    "Correlation Coefficients"
]

df["cluster"] = pd.Categorical(df["cluster"], categories=desired_order, ordered=True)
df = df.sort_values("cluster")

# Create the 2-line text (keep <br> for inside; outside works too)
df["slice_text_multiline"] = df.apply(
    lambda r: f"{int(r['n_metrics'])}<br>({r['percentage']:.1f}%)",
    axis=1
)

# --- choose when to push labels outside ---
# Anything below this percent goes outside (tune 4–8 depending on crowding)
OUTSIDE_THRESHOLD_PCT = 6.0

is_small = df["percentage"] < OUTSIDE_THRESHOLD_PCT

# Pull out only the small slices (helps leader lines + avoids collisions)
pull = np.where(is_small, 0.08, 0.0)

fig = px.pie(
    df,
    names="cluster",
    values="n_metrics",
    color="cluster",
    color_discrete_map=cluster_colors,
    category_orders={"cluster": desired_order}
)

fig.update_traces(
    sort=False,
    text=df["slice_text_multiline"],
    textinfo="text",
    textfont=dict(size=22),  # ✅ force label font size

    # ✅ mixed strategy:
    textposition=np.where(is_small, "outside", "inside"),

    # inside label orientation
    insidetextorientation="horizontal",

    # help outside labels with leader lines
    pull=pull,

    hovertemplate=(
        "<b>%{label}</b><br>"
        "N = %{value}<br>"
        "% = %{percent:.1%}<extra></extra>"
    ),
)

fig.update_layout(
    title=None,
    showlegend=True,
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=10, b=10),
    font=dict(size=26),

    # ✅ IMPORTANT: do NOT allow Plotly to shrink text
    #uniformtext_minsize=24,
    #uniformtext_mode="show"
)

fig.show()
fig.write_image("Fig1_Metrics_overview2.png", scale=3)
fig.write_image("Fig1_Metrics_overview2.svg", scale=3)
fig.write_image("Fig1_Metrics_overview2.pdf", scale=3)
# ==========================

# 2. Overview (Number of metrics per dimension):

In [18]:
import pandas as pd
import plotly.express as px

# ==========================
# 0) Inputs
# ==========================
file_path = "MetricLibrary1.xlsx"          # <-- change
sheet_name = "Metrics_dimensions"          # <-- change

cluster_colors = {
    "Measurement Process":      "#9C9EFF",
    "Consistency":              "#C8BAF3",
    "Representativeness":       "#77EBA8",
    "Informativeness":          "#51AAFF",
    "Timeliness":               "#73D3FC"
}

desired_order = [
    "Measurement Process",
    "Consistency",
    "Representativeness",
    "Informativeness",
    "Timeliness"
]

# ==========================
# 1) Load & prep data
# ==========================
df = pd.read_excel(file_path, sheet_name=sheet_name)

df = df.rename(columns={
    "Cluster": "cluster",
    "Cluster name": "cluster",
    "Cluster Name": "cluster",
    "Dimension": "dimension",
    "DImension": "dimension",
    "DImnsion": "dimension",
    "Number of metrics dimension": "n_dim",
    "Number of metrics dimensions": "n_dim",
})

df = df[["cluster", "dimension", "n_dim"]].copy()

df["cluster"] = df["cluster"].astype(str).str.strip()
df["dimension"] = df["dimension"].astype(str).str.strip()
df["n_dim"] = pd.to_numeric(df["n_dim"], errors="coerce").fillna(0)

df = df[df["cluster"].isin(desired_order)].copy()

df_plot = (
    df.groupby(["cluster", "dimension"], as_index=False)["n_dim"]
      .sum()
)

cluster_rank = {c: i for i, c in enumerate(desired_order)}
df_plot["cluster_rank"] = df_plot["cluster"].map(cluster_rank)

df_plot = df_plot.sort_values(
    ["cluster_rank", "n_dim", "dimension"],
    ascending=[True, False, True]
)

dimension_order = df_plot["dimension"].tolist()

# ==========================
# 2) Vertical bar chart
# ==========================
px.defaults.template = "simple_white"

fig = px.bar(
    df_plot,
    x="dimension",
    y="n_dim",
    color="cluster",
    color_discrete_map=cluster_colors,
    category_orders={
        "cluster": desired_order,
        "dimension": dimension_order
    },
    text="n_dim",
)

# --- sizes (edit as you like) ---
AXIS_TICK_SIZE = 22
AXIS_TITLE_SIZE = 24
ANNOTATION_SIZE = 22

fig.update_traces(
    textposition="outside",
    textfont=dict(size=ANNOTATION_SIZE),  # value labels ("annotations" on bars)
)

fig.update_layout(
    title=None,
    width=800,
    height=500,
    xaxis_title=None,
    yaxis_title="Number of metrics in dimension",
    #yaxis range
    yaxis=dict(range=[0, df_plot["n_dim"].max() * 1.15]),
    showlegend=False,  # <-- remove legend
    margin=dict(l=60, r=30, t=60, b=200),
    font=dict(size=AXIS_TICK_SIZE),  # baseline font size
)

# Bigger tick labels + axis title
fig.update_xaxes(
    tickangle=45,
    tickfont=dict(size=AXIS_TICK_SIZE),
    title_font=dict(size=AXIS_TITLE_SIZE),
)
fig.update_yaxes(
    tickfont=dict(size=AXIS_TICK_SIZE),
    title_font=dict(size=AXIS_TITLE_SIZE),
)

fig.show()
fig.write_html("Fig_dimension_counts_by_cluster_horizontal.html")
print("Saved: Fig_dimension_counts_by_cluster_horizontal.html")

# Optional exports
fig.write_image("Fig_dimension_counts_by_cluster_horizontal.png", scale=3)
fig.write_image("Fig_dimension_counts_by_cluster_horizontal.svg")
fig.write_image("Fig_dimension_counts_by_cluster_horizontal.pdf")


Saved: Fig_dimension_counts_by_cluster_horizontal.html


In [19]:
# ==========================
# 0) Inputs
# ==========================
file_path = "MetricLibrary1.xlsx"
sheet_name = "Metrics_dimensions"

cluster_colors = {
    "Measurement Process": "#9C9EFF",
    "Consistency": "#C8BAF3",
    "Representativeness": "#77EBA8",
    "Informativeness": "#51AAFF",
    "Timeliness": "#73D3FC"
}

desired_order = [
    "Measurement Process",
    "Consistency",
    "Representativeness",
    "Informativeness",
    "Timeliness"
]

# ==========================
# 1) Load & prep data
# ==========================
df = pd.read_excel(file_path, sheet_name=sheet_name)

df = df.rename(columns={
    "Cluster": "cluster",
    "Cluster name": "cluster",
    "Cluster Name": "cluster",
    "Dimension": "dimension",
    "DImension": "dimension",
    "DImnsion": "dimension",
    "Number of metrics dimension": "n_dim",
    "Number of metrics dimensions": "n_dim",
})

df = df[["cluster", "dimension", "n_dim"]].copy()

df["cluster"] = df["cluster"].astype(str).str.strip()
df["dimension"] = df["dimension"].astype(str).str.strip()
df["n_dim"] = pd.to_numeric(df["n_dim"], errors="coerce").fillna(0)

df = df[df["cluster"].isin(desired_order)].copy()

df_plot = (
    df.groupby(["cluster", "dimension"], as_index=False)["n_dim"]
      .sum()
)

cluster_rank = {c: i for i, c in enumerate(desired_order)}
df_plot["cluster_rank"] = df_plot["cluster"].map(cluster_rank).fillna(999)

df_plot = df_plot.sort_values(
    ["cluster_rank", "n_dim", "dimension"],
    ascending=[True, False, True]
)

df_plot["y_label"] = df_plot["dimension"]

# ==========================
# 2) Horizontal bar chart
# ==========================
px.defaults.template = "simple_white"

fig = px.bar(
    df_plot,
    x="n_dim",
    y="y_label",
    orientation="h",
    color="cluster",
    color_discrete_map=cluster_colors,
    category_orders={"cluster": desired_order},
    text="n_dim",
)

# ---- FONT SIZES ----
AXIS_TICK_SIZE = 26
AXIS_TITLE_SIZE = 26
ANNOTATION_SIZE = 26

fig.update_traces(
    textposition="outside",
    textfont=dict(size=ANNOTATION_SIZE),
    cliponaxis=False
)

fig.update_layout(
    title=None,
    width=800,
    height=800,
    showlegend=False,   # ✅ legend removed
    margin=dict(l=250, r=40, t=30, b=30),
    font=dict(size=AXIS_TICK_SIZE),
)

fig.update_xaxes(
    title="Number of metrics in dimension",
    tickmode="linear",
    tick0=0,
    dtick=10,
    tickfont=dict(size=AXIS_TICK_SIZE),
    title_font=dict(size=AXIS_TITLE_SIZE),
)

fig.update_yaxes(
    title=None,
    tickfont=dict(size=AXIS_TICK_SIZE),
    autorange="reversed"   # top-to-bottom follows desired_order
)

fig.show()

fig.write_html("Fig_dimension_counts_by_cluster_vertical1.html")
fig.write_image("Fig_dimension_counts_by_cluster_vertical1.png", scale=3)
fig.write_image("Fig_dimension_counts_by_cluster_vertical1.svg")
fig.write_image("Fig_dimension_counts_by_cluster_vertical1.pdf")

print("Saved: Fig_dimension_counts_by_cluster_vertical1")


Saved: Fig_dimension_counts_by_cluster_vertical1


## 3. Sankey plots (metrics dimension cluster)

In [40]:
import pandas as pd
import plotly.graph_objects as go

# ==========================
# 0. Inputs
# ==========================
file_path = "MetricLibrary3.xlsx"     # <-- change
sheet_name = "senky_1"               # <-- change

cluster_colors = {
    "Measurement Process":      "#9C9EFF",
    "Consistency":              "#C8BAF3",
    "Representativeness":       "#77EBA8",
    "Informativeness":          "#51AAFF",
    "Timeliness":               "#73D3FC",
    "Distribution Metrics cluster":     "#DB9ECF",
    "Correlation Coefficients cluster": "#F79B7C"
}

desired_dimension_order = [
    "Accuracy",
    "Noisy labels",
    "Completeness",
    "Distribution drift",
    "Syntactic Consistency",
    "Target class balance",
    "Dataset size",
    "Granularity",
    "Currency",
    "Uniqueness",
    "Informative missingness",
    "Distribution metrics",
    "Correlation Coefficients",
]

datatype_cols = ["Image", "Time-series", "Tabular", "Text", "Multimodal"]

# ==========================
# 1. Load data
# ==========================
df = pd.read_excel(file_path, sheet_name=sheet_name)

df = df.rename(columns={
    "Cluster name": "cluster",
    "Cluster Name": "cluster",
    "Cluster": "cluster",
    "Number of metrics cluster": "n_cluster",
    "Number of metrics per cluster": "n_cluster",
    "Number of metrics cluster ": "n_cluster",
    "Dimension": "dimension",
    "DImnsion": "dimension",
    "DImension": "dimension",
    "Number of metrics dimensions": "n_dimension",
    "Number of metrics dimension": "n_dimension",
    "Number of metrics per dimension": "n_dimension",
})

keep_cols = ["cluster", "dimension"] + datatype_cols
missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in Excel: {missing}")

df = df[keep_cols].copy()

df["cluster"] = df["cluster"].astype(str).str.strip()
df["dimension"] = df["dimension"].astype(str).str.strip()
for c in datatype_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

df = df[(df["dimension"] != "") & (df["dimension"].str.lower() != "nan")]
df = df[(df["cluster"] != "") & (df["cluster"].str.lower() != "nan")]

# ==========================
# 2. Build nodes (TWO columns only)
#    Dimensions → Datatypes
# ==========================
dimensions_present = set(df["dimension"].unique())
dimensions = [d for d in desired_dimension_order if d in dimensions_present]
datatypes = datatype_cols[:]

nodes = dimensions + datatypes
node_index = {n: i for i, n in enumerate(nodes)}

# Node colors: keep neutral for dims + datatypes (colors are preserved on LINKS via cluster color)
node_colors = (
    ["#DDDDDD" for _ in dimensions] +
    ["#CCCCCC" for _ in datatypes]
)

# ==========================
# 3. Build links (Dimension → Data type), colored by cluster
# ==========================
sources, targets, values, link_colors = [], [], [], []

for _, r in df.iterrows():
    dim = r["dimension"]
    cl = r["cluster"]

    if dim not in node_index:
        continue

    for dt in datatypes:
        v = float(r[dt])
        if v <= 0:
            continue
        if dt not in node_index:
            continue

        sources.append(node_index[dim])
        targets.append(node_index[dt])
        values.append(v)

        # KEEP original cluster colors (same as your 3-column version)
        col = cluster_colors.get(cl, "#999999")
        link_colors.append(
            f"rgba({int(col[1:3],16)},{int(col[3:5],16)},{int(col[5:7],16)},0.30)"
        )

# ==========================
# 4. Fix node positions (increase vertical spacing for DIMENSIONS)
# ==========================
n_dimensions = len(dimensions)
n_datatypes = len(datatypes)

def spaced_y(n, top=0.02, bottom=0.98):
    if n <= 1:
        return [0.5]
    step = (bottom - top) / (n - 1)
    return [top + i * step for i in range(n)]

# Give dimensions more vertical room by tightening top/bottom less (more spread feel)
y_dimensions = spaced_y(n_dimensions, top=0.02, bottom=0.98)
y_datatypes  = spaced_y(n_datatypes,  top=0.08, bottom=0.92)  # slightly tighter on right

# Two x columns only
x_dimensions = [0.01] * n_dimensions
x_datatypes  = [0.99] * n_datatypes

node_x = x_dimensions + x_datatypes
node_y = y_dimensions + y_datatypes

# ==========================
# 5. Build Sankey (bigger text + bigger spacing)
# ==========================
FONT_SIZE = 24       # bigger text
NODE_PAD = 60         # increases vertical distance between nodes
NODE_THICKNESS = 18

fig = go.Figure(go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=NODE_PAD,
        thickness=NODE_THICKNESS,
        line=dict(color="white", width=1),
        label=nodes,
        color=node_colors,
        x=node_x,
        y=node_y
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors,
        hovertemplate="From %{source.label} → %{target.label}<br>Count: %{value}<extra></extra>"
    )
))

fig.update_layout(
    title_text=None,
    font_size=FONT_SIZE,
    width=800,
    height=1600,   # taller to visually increase spacing between dimension nodes
    margin=dict(t=50, r=30, b=50, l=30)
)

fig.show()
fig.write_html("sankey_dimension_datatype_ordered.html")
print("Saved: sankey_dimension_datatype_ordered.html")
#300 dpi
fig.write_image("Fig3_Sankey_dimension_datatype_ordered.png", width=800, height=1600, scale=300/72)
fig.write_image("Fig3_Sankey_dimension_datatype_ordered.svg",scale=3)
fig.write_image("Fig3_Sankey_dimension_datatype_ordered.pdf", scale=3)
print("Saved: Fig3_Sankey_dimension_datatype_ordered (png/svg/pdf)")


Saved: sankey_dimension_datatype_ordered.html
Saved: Fig3_Sankey_dimension_datatype_ordered (png/svg/pdf)


In [49]:
import pandas as pd
import plotly.graph_objects as go

# ==========================
# 0. Inputs
# ==========================
file_path = "MetricLibrary3.xlsx"   # <-- change if needed
sheet_name = "senky_1"             # <-- change if needed

cluster_colors = {
    "Measurement Process":      "#9C9EFF",
    "Consistency":              "#C8BAF3",
    "Representativeness":       "#77EBA8",
    "Informativeness":          "#51AAFF",
    "Timeliness":               "#73D3FC",
    "Distribution Metrics cluster":     "#DB9ECF",
    "Correlation Coefficients cluster": "#F79B7C"
}

desired_dimension_order = [
    "Accuracy",
    "Noisy labels",
    "Completeness",
    "Distribution drift",
    "Syntactic Consistency",
    "Target class balance",
    "Dataset size",
    "Granularity",
    "Currency",
    "Uniqueness",
    "Informative missingness",
    "Distribution metrics",
    "Correlation Coefficients",
]

datatype_cols = ["Image", "Time-series", "Tabular", "Text", "Multimodal"]

# ==========================
# 1. Load data
# ==========================
df = pd.read_excel(file_path, sheet_name=sheet_name)

df = df.rename(columns={
    "Cluster name": "cluster",
    "Cluster Name": "cluster",
    "Cluster": "cluster",
    "Number of metrics cluster": "n_cluster",
    "Number of metrics per cluster": "n_cluster",
    "Number of metrics cluster ": "n_cluster",
    "Dimension": "dimension",
    "DImnsion": "dimension",
    "DImension": "dimension",
    "Number of metrics dimensions": "n_dimension",
    "Number of metrics dimension": "n_dimension",
    "Number of metrics per dimension": "n_dimension",
})

keep_cols = ["cluster", "dimension"] + datatype_cols
missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in Excel: {missing}")

df = df[keep_cols].copy()

df["cluster"] = df["cluster"].astype(str).str.strip()
df["dimension"] = df["dimension"].astype(str).str.strip()
for c in datatype_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

df = df[(df["dimension"] != "") & (df["dimension"].str.lower() != "nan")]
df = df[(df["cluster"] != "") & (df["cluster"].str.lower() != "nan")]

# ==========================
# 2. Build nodes (TWO columns only)
#    Dimensions → Datatypes
#    (ORDER IS PRESERVED via desired_dimension_order)
# ==========================
dimensions_present = set(df["dimension"].unique())
dimensions = [d for d in desired_dimension_order if d in dimensions_present]
datatypes = datatype_cols[:]

nodes = dimensions + datatypes
node_index = {n: i for i, n in enumerate(nodes)}

# Node colors: neutral for dims + datatypes (link colors encode cluster)
node_colors = (["#DDDDDD" for _ in dimensions] + ["#CCCCCC" for _ in datatypes])

# ==========================
# 3. Build links (Dimension → Data type), colored by cluster
# ==========================
sources, targets, values, link_colors = [], [], [], []

for _, r in df.iterrows():
    dim = r["dimension"]
    cl = r["cluster"]

    if dim not in node_index:
        continue

    for dt in datatypes:
        v = float(r[dt])
        if v <= 0:
            continue
        if dt not in node_index:
            continue

        sources.append(node_index[dim])
        targets.append(node_index[dt])
        values.append(v)

        col = cluster_colors.get(cl, "#999999")
        link_colors.append(
            f"rgba({int(col[1:3],16)},{int(col[3:5],16)},{int(col[5:7],16)},0.30)"
        )

# ==========================
# 4. Fix node positions
#    OPTION B: locally spread ONLY the bottom few dimension nodes
#    (keeps dimension order the same)
# ==========================
n_dimensions = len(dimensions)
n_datatypes = len(datatypes)

def spaced_y(n, top=0.02, bottom=0.98):
    if n <= 1:
        return [0.5]
    step = (bottom - top) / (n - 1)
    return [top + i * step for i in range(n)]

y_dimensions = spaced_y(n_dimensions, top=0.02, bottom=0.98)
y_datatypes  = spaced_y(n_datatypes,  top=0.08, bottom=0.92)

# ---- OPTION B PARAMETERS ----
EXTRA_GAP = 0.01   # try 0.03–0.08 (bigger = more separation at bottom)
K = 2              # spread bottom K dimension nodes (incl. last two)

# Spread the bottom K dimension nodes downward progressively
K = min(K, n_dimensions)
for i in range(1, K + 1):
    idx = -i
    # last node gets the biggest push, then slightly less above it
    y_dimensions[idx] = min(0.98, y_dimensions[idx] + EXTRA_GAP * (K - i + 1) / K)

# Safety: ensure strictly increasing y (prevents overlaps from ties)
MIN_DY = 0.002
for i in range(1, len(y_dimensions)):
    if y_dimensions[i] <= y_dimensions[i - 1] + MIN_DY:
        y_dimensions[i] = y_dimensions[i - 1] + MIN_DY

# Clamp again to bounds
y_dimensions = [min(0.98, max(0.02, y)) for y in y_dimensions]

# Two x columns only
x_dimensions = [0.01] * n_dimensions
x_datatypes  = [0.99] * n_datatypes

node_x = x_dimensions + x_datatypes
node_y = y_dimensions + y_datatypes

# ==========================
# 5. Build Sankey
# ==========================
FONT_SIZE = 26
NODE_PAD = 80
NODE_THICKNESS = 18

fig = go.Figure(go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=NODE_PAD,
        thickness=NODE_THICKNESS,
        line=dict(color="white", width=1),
        label=nodes,
        color=node_colors,
        x=node_x,
        y=node_y
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors,
        hovertemplate="From %{source.label} → %{target.label}<br>Count: %{value}<extra></extra>"
    )
))

fig.update_layout(
    title_text=None,
    font_size=FONT_SIZE,
    width=800,
    height=1600,
    margin=dict(t=50, r=30, b=50, l=30)
)

fig.show()

fig.write_html("sankey_dimension_datatype_ordered.html")
print("Saved: sankey_dimension_datatype_ordered.html")

# 300 dpi PNG
fig.write_image(
    "Fig3_Sankey_dimension_datatype_ordered.png",
    width=800,
    height=1600,
    scale=300/72
)
fig.write_image("Fig3_Sankey_dimension_datatype_ordered.svg", scale=3)
fig.write_image("Fig3_Sankey_dimension_datatype_ordered.pdf", scale=3)

print("Saved: Fig3_Sankey_dimension_datatype_ordered (png/svg/pdf)")


Saved: sankey_dimension_datatype_ordered.html
Saved: Fig3_Sankey_dimension_datatype_ordered (png/svg/pdf)
